In [ ]:
import omnipath as op
import pandas as pd
import networkx as nx
import numpy as np
import scipy
import torch
import torch_geometric
from torch_geometric.data import Data
import sklearn
from run.run_undirected_scattering import *

## Run Undirected Scattering on training graph

In [ ]:
for dataset in ['Texas', 'Cornell', 'omnipath', 'SIGNOR', 'iPTMnet']:
    print (dataset)
    
    splits = np.load(f'data/{dataset}_curated_interactions.npz')
    train, val, test = splits['train'], splits['val'], splits['test']
    train.columns = ['source', 'target']

    G = nx.from_pandas_edgelist(train, source='source', target='target')
    A = nx.adjacency_matrix(G).toarray()
    N = A.shape[0]
    
    GS = graph_scattering(A, scales=range(2), q = 0.1, order= 2)
    GS.dirac_signals()
    GS.diffusion_operators()
    GS.zeroth_order_transform()
    GS.first_order_transform()
    GS.second_order_transform()

    gz = GS.zeroth_order.reshape((1,GS.zeroth_order.shape[0], GS.zeroth_order.shape[1]))
    g1 = GS.first_order
    g2 = GS.second_order
    z = np.concatenate([gz, g1, g2], axis = 0)

    c = np.transpose(z, (1,0,2)) #to take moments by filter scale
    mean, variance, skew, kurtosis  = calculate_stats(c)
    stats = np.concatenate([mean, variance, skew, kurtosis])
    vals = pd.DataFrame(stats.T).fillna(0).values
    pd.DataFrame(vals, index=G.nodes).to_csv(f'results/Undirected_Scattering/Undirected_Scattering_{dataset}_embedding.csv')

## Run Undirected Scattering on train+val graph

In [ ]:
for dataset in ['Texas', 'Cornell', 'omnipath', 'SIGNOR', 'iPTMnet']:
    print (dataset)
    
    splits = np.load(f'data/{dataset}_curated_interactions.npz')
    train, val, test = splits['train'], splits['val'], splits['test']
    train = pd.DataFrame(np.vstack((train, val)))
    train.columns = ['source', 'target']

    G = nx.from_pandas_edgelist(train, source='source', target='target')
    A = nx.adjacency_matrix(G).toarray()
    N = A.shape[0]
    
    GS = graph_scattering(A, scales=range(2), q = 0.1, order= 2)
    GS.dirac_signals()
    GS.diffusion_operators()
    GS.zeroth_order_transform()
    GS.first_order_transform()
    GS.second_order_transform()

    gz = GS.zeroth_order.reshape((1,GS.zeroth_order.shape[0], GS.zeroth_order.shape[1]))
    g1 = GS.first_order
    g2 = GS.second_order
    z = np.concatenate([gz, g1, g2], axis = 0)

    c = np.transpose(z, (1,0,2)) #to take moments by filter scale
    mean, variance, skew, kurtosis  = calculate_stats(c)
    stats = np.concatenate([mean, variance, skew, kurtosis])
    vals = pd.DataFrame(stats.T).fillna(0).values
    pd.DataFrame(vals, index=G.nodes).to_csv(f'results/Undirected_Scattering/Undirected_Scattering_{dataset}_train_val_embedding.csv')